# Advanced CNN Architecture

## Scenes Classification

## 1.Tải bộ dữ liệu

In [1]:
# bộ dữ liệu mg_cls_scenes_classification.zip
# https://drive.google.com/file/d/1ZUCuYDOe4VVbZvNVZovpquaRQqqJQ639/view?usp=drive_link
%cd /content/drive/MyDrive/Colab Notebooks/MODULE06/W02
# !gdow 1ZUCuYDOe4VVbZvNVZovpquaRQqqJQ639

[WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/Colab Notebooks/MODULE06/W02'
g:\AIO\EXERCISE\AIO-2024\MODULE06\WEEK02


## 2.Import các thư viện cần thiết

In [2]:
import torch
import torch.nn as nn
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

## 3.Cố định giá trị ngẫu nhiên
Để có thể tái tạo lại cùng một kết quả mô hình, chúng ta sẽ cố định cùng một giá trị ngẫu nhiên (seed) cho các thư viện có chứa các hàm tạo giá trị ngẫu nhiên

In [ ]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

seed = 59
set_seed(seed)

## 4.Đọc dữ liệu
Để thuận tiện trong việc xây dựng PyTorch datasets, chúng ta sẽ ghi nhận thông tin về các classes, đường dẫn đến tất cả các ảnh cũng như label tương ứng như sau.

In [ ]:
%pwd

'/content/drive/MyDrive/Colab Notebooks/MODULE06/W02'

In [ ]:
root_dir = 'Data/scenes_classification'
train_dir = os.path.join(root_dir, 'train')
test_dir = os.path.join(root_dir, 'val')
classes = {label_idx:class_name for label_idx, class_name in enumerate(sorted(os.listdir(train_dir)))}

In [ ]:
classes.keys(), classes.values()

(dict_keys([0, 1, 2, 3, 4, 5]),
 dict_values(['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']))

In [ ]:
classes

{0: 'buildings',
 1: 'forest',
 2: 'glacier',
 3: 'mountain',
 4: 'sea',
 5: 'street'}

In [ ]:
# Đọc lên toàn bộ các đường dẫn ảnh cũng như label tương ứng. Tuy nhiên, ta sẽ coi thư mục val là tập test của bộ dữ liệu và sẽ khai báo danh sách riêng cho bộ này
X_train = []
y_train = []
X_test = []
y_test = []
for dataset_path in [train_dir, test_dir]:
  for label_idx, class_name in classes.items():
    class_dir = os.path.join(dataset_path, class_name)
    for img_filename in os.listdir(class_dir):
      img_path = os.path.join(class_dir, img_filename)
      if 'train' in dataset_path:
        X_train.append(img_path)
        y_train.append(label_idx)
      else:
        X_test.append(img_path)
        y_test.append(label_idx)


In [ ]:
len(X_train), len(y_train), len(X_test), len(y_test)

(14034, 14034, 3000, 3000)

In [ ]:
y_train[:10], y_test[:10]

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
X_train[:10], X_test[:10]

(['Data/scenes_classification/train/buildings/19447.jpg',
  'Data/scenes_classification/train/buildings/19460.jpg',
  'Data/scenes_classification/train/buildings/19440.jpg',
  'Data/scenes_classification/train/buildings/19526.jpg',
  'Data/scenes_classification/train/buildings/19513.jpg',
  'Data/scenes_classification/train/buildings/1964.jpg',
  'Data/scenes_classification/train/buildings/19416.jpg',
  'Data/scenes_classification/train/buildings/1943.jpg',
  'Data/scenes_classification/train/buildings/1954.jpg',
  'Data/scenes_classification/train/buildings/19556.jpg'],
 ['Data/scenes_classification/val/buildings/20078.jpg',
  'Data/scenes_classification/val/buildings/20207.jpg',
  'Data/scenes_classification/val/buildings/20057.jpg',
  'Data/scenes_classification/val/buildings/20083.jpg',
  'Data/scenes_classification/val/buildings/20113.jpg',
  'Data/scenes_classification/val/buildings/20073.jpg',
  'Data/scenes_classification/val/buildings/20186.jpg',
  'Data/scenes_classification/

## 5.Chia bộ dữ liệu train, val, test
Vì đã có sẵn bộ dữ liệu train và test, ta chỉ việc
chia thêm cho tập val từ tập train

In [ ]:
seed = 0
val_size = 0.2
is_shuffle = True
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=seed, shuffle=is_shuffle)

In [ ]:
len(X_train), len(X_val), len(X_test)

(11227, 2807, 3000)

In [ ]:
X_train[:10], X_val[:10], X_test[:10]

(['Data/scenes_classification/train/buildings/11523.jpg',
  'Data/scenes_classification/train/glacier/15501.jpg',
  'Data/scenes_classification/train/glacier/1910.jpg',
  'Data/scenes_classification/train/forest/11068.jpg',
  'Data/scenes_classification/train/sea/8062.jpg',
  'Data/scenes_classification/train/mountain/7226.jpg',
  'Data/scenes_classification/train/mountain/10323.jpg',
  'Data/scenes_classification/train/mountain/7113.jpg',
  'Data/scenes_classification/train/glacier/7502.jpg',
  'Data/scenes_classification/train/street/18005.jpg'],
 ['Data/scenes_classification/train/glacier/19920.jpg',
  'Data/scenes_classification/train/buildings/16692.jpg',
  'Data/scenes_classification/train/street/10729.jpg',
  'Data/scenes_classification/train/street/3298.jpg',
  'Data/scenes_classification/train/mountain/10666.jpg',
  'Data/scenes_classification/train/sea/7233.jpg',
  'Data/scenes_classification/train/glacier/19197.jpg',
  'Data/scenes_classification/train/forest/18907.jpg',
  '

## 6.Xây dựng class pytorch datasets
xây dựng pytorch dataset cho bộ dữ liệu Scenes

In [ ]:
class ScenesDataset(Dataset):
  def __init__(self, X, y, transform=None):
    self.transform = transform
    self.img_paths = X
    self.labels = y

  def __len__(self):
    return len(self.img_paths)

  def __getitem__(self, idx):
    img_path = self.img_paths[idx]
    img = Image.open(img_path).convert("RGB")

    if self.transform:
      img = self.transform(img)

    return img, self.labels[idx]

## 7.Xây dựng hàm tiền xử lý ảnh (transform)
Để đảm bảo dữ liệu ảnh đầu vào được đồng bộ về kích thước và giá trị, chúng ta tự định nghĩa hàm transform để tiền xử lý ảnh đầu vào như sau (không sử dụng thư viện torchvision.transforms). Các kỹ thuật được áp dụng: resize ảnh, đổi về tensor và chuẩn hóa giá trị pixel về khoảng (0, 1)

In [ ]:
def transform (img, img_size =(224, 224)):
  img = img.resize(img_size)
  img = np.array(img)[... , :3]
  img = torch.tensor(img).permute(2, 0, 1).float()
  normalized_img = img/255.0

  return normalized_img

## 8.Khai báo datasets object cho ba bộ train, val, test
khai báo ba object datasets tương ứng cho ba bộ dữ liệu train, val, test

In [ ]:
train_dataset = ScenesDataset(X_train, y_train,transform=transform)
val_dataset = ScenesDataset(X_val, y_val,transform=transform)
test_dataset = ScenesDataset(X_test, y_test,transform=transform)

In [ ]:
len(train_dataset), len(val_dataset), len(test_dataset)

(11227, 2807, 3000)

## 9.Khai báo dataloader
Với ba object datasets trên, ta khai báo giá trị batch size và
tạo dataloader như sau

In [ ]:
train_batch_size = 64
test_batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=test_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

In [ ]:
train_loader

## 10.Xây dựng model
xây dựng class cho model deep learning với kiến trúc DenseNet.

In [ ]:
#xây dựng class Dense Block, đây là một thành phần đặc biệt của kiến trúc DenseNet
class BottleneckBlock (nn.Module):
  def __init__(self, in_channels, growth_rate):
    super(BottleneckBlock, self).__init__()
    self.bn1 = nn.BatchNorm2d(in_channels)
    self.conv1 = nn.Conv2d(in_channels, 4*growth_rate, kernel_size=1, bias=False)
    self.bn2 = nn.BatchNorm2d(4*growth_rate)
    self.conv2 = nn.Conv2d(4*growth_rate, growth_rate,kernel_size=3, padding=1, bias=False)
    self.relu = nn.ReLU()

  def forward(self, x):
    res = x.clone().detach()
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv1(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = torch.cat([res, x], 1)

    return x

class DenseBlock(nn.Module):
  def __init__(self, num_layers, in_channels, growth_rate):
    super(DenseBlock, self).__init__()
    layers = []
    for i in range(num_layers):
      layers.append(BottleneckBlock(in_channels + i*growth_rate, growth_rate))

    self.block = nn.Sequential(*layers)

  def forward(self, x):
    return self.block(x)

In [ ]:
#Với DenseBlock, ta triển khai toàn bộ kiến trúc DenseNet
class DenseNet(nn.Module):
  def __init__(self, num_blocks, growth_rate, num_classes):
    super(DenseNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 2*growth_rate, kernel_size=7, padding=3, stride=2, bias=False)
    self.bn1 = nn.BatchNorm2d(2*growth_rate)
    self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.dense_blocks = nn.ModuleList()
    in_channels = 2*growth_rate

    for i, num_layers in enumerate(num_blocks):
      self.dense_blocks.append(DenseBlock(num_layers, in_channels, growth_rate))
      in_channels += num_layers*growth_rate
      if i != len(num_blocks) - 1:
        out_channels = in_channels//2
        self.dense_blocks.append(nn.Sequential(
                                              nn.BatchNorm2d(in_channels),
                                              nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
                                              nn.AvgPool2d(kernel_size=2, stride=2)
                                              ))
        in_channels = out_channels

    self.bn2 = nn.BatchNorm2d(in_channels)
    self.pool2 = nn.AvgPool2d(kernel_size=7)
    self.relu = nn.ReLU()
    self.fc = nn.Linear(in_channels,num_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.pool1(x)

    for block in self.dense_blocks:
      x = block(x)

    x = self.bn2(x)
    x = self.relu(x)
    x = self.pool2(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)

    return x #self.block(x)

In [ ]:
#khai báo model DenseNet
n_classes = len(list(classes.keys()))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = DenseNet([6, 12, 24, 16], growth_rate=32, num_classes=n_classes).to(device)

In [ ]:
device

'cuda'

In [ ]:
n_classes

6

## 11.Khai báo hàm loss và thuật toán huấn luyện
sử dụng hàm loss CrossEntropy và Stochastic Gradient

In [ ]:
lr = 1e-2
epochs = 15
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
def evaluate(model, dataloader, criterion, device):
  model.eval()
  correct = 0
  total = 0
  losses = []
  with torch.no_grad():
    for inputs, labels in dataloader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      losses.append(loss.item())
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      loss = sum(losses)/len(losses)
      acc = correct/total
  return loss, acc

In [ ]:
def fit(model, train_loader, val_loader, criterion, optimizer, device, epochs):
  train_losses = []
  val_losses = []
  for epoch in range(epochs):
    batch_train_losses = []

    model.train()
    for idx, (inputs, labels) in enumerate(train_loader):
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      batch_train_losses.append(loss.item())

    train_loss = sum(batch_train_losses)/len(batch_train_losses)
    train_losses.append(train_loss)

    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    val_losses.append(val_loss)

    print (f'EPOCH{epoch+1}:\tTrain loss:{train_loss:.4f}\tVal loss:{val_loss:.4f}')

  return train_losses , val_losses

## 12.Xây dựng hàm huấn luyện model

In [ ]:
model

DenseNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dense_blocks): ModuleList(
    (0): DenseBlock(
      (block): Sequential(
        (0): BottleneckBlock(
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (relu): ReLU()
        )
        (1): BottleneckBlock(
          (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv1): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [ ]:
train_loader, val_loader, criterion, optimizer, device, epochs

(<torch.utils.data.dataloader.DataLoader at 0x7d4be26a9de0>,
 CrossEntropyLoss(),
 SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     fused: None
     lr: 0.01
     maximize: False
     momentum: 0
     nesterov: False
     weight_decay: 0
 ),
 'cuda',
 15)

In [ ]:
train_losses, val_losses = fit(model, train_loader, val_loader, criterion, optimizer, device, epochs)

## 13.Đánh giá model
Ta gọi hàm evaluate() để đánh giá performance của model trên
hai tập val và test

In [ ]:
val_loss, val_acc = evaluate(model, val_loader, criterion, device)
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print('Evaluation on val/ test dataset')
print('Val accuracy:', val_acc)
print('Test accuracy:', test_acc)

In [ ]:
#  trực quan kết quả huấn luyện trên tập train và val cho bài toán nonlinear data classification
fig , ax = plt.subplots(1, 2, figsize=(12, 10))

ax[0, 0].plot(train_losses, color='green')
ax[0, 0].set(xlabel='Epoch', ylabel='Loss')
ax[0, 0].set_title('Training Loss')

ax [0, 1].plot(val_losses, color='orange')
ax [0, 1].set(xlabel='Epoch', ylabel='Loss')
ax [0, 1].set_title('Validation Loss')


plt.show()